In [1]:
%serialconnect --port=COM7

Connecting to --port=COM7 --baud=115200 
Ready.


In [2]:
# import mip
# import network
# sta = network.WLAN(network.STA_IF)
# sta.active(True)
# sta.connect("West3")
# sta.ifconfig()

# mip.install("https://gist.githubusercontent.com/shawwwn/91cc8979e33e82af6d99ec34c38195fb/raw/ca2e629a54abcb18b1c4f766d594507cea41289a/uping.py")

In [3]:
from machine import Pin, SoftI2C
from time import sleep 
import ntptime
import network
import esp32
import ubinascii
import time
import espnow
import gc
import ssd1306Driver
import machine
from uping import ping

In [4]:
class ESP32_Wroom32Devkit1: 
    """
       DevKit1 Pin out URL:  https://www.circuitstate.com/pinouts/doit-esp32-devkit-v1-wifi-development-board-pinout-diagram-and-reference/#!fancybox/06b225e2/ESP32-DevKit-V1-Pinout-Diagram-r0.1-CIRCUITSTATE-Electronics-2.png
    """
    def __init__(self, verbose = True):
        from machine import Pin, SoftI2C
        from time import sleep 
        import network
        import esp32
        import gc
        self.onboardLedGPIO = 2
        self.CpuTemperature = 0
        self.led = None
        self.wlanMacHex = None
        self.wlanMacAscii = None
        self.unquieId =  None
        self.wifiStation = None
        self.freeMemory = None
        self.oled = None
        self.espNowObj = None
        self.verbose = verbose
        self.wifiChannel = None
        #Pins 
        self.sdaPin = 21
        self.sclPin = 22
        #Update RTC 
        
        
    def getNtpTime(self):
        host = "pool.ntp.org"
        try:
            ntptime.host = host
            ntptime.settime()
            print("Time set to:", machine.RTC().datetime())
        except (OSError, OverflowError) as e:
            print("ntptime.settime():", e)
    def getFreeMemory(self):
        self.freeMemory = gc.mem_free()/1024
        if(self.verbose): print(f"Free Memory : {self.freeMemory}")
        return(self.freeMemory)
    def getUniqueId(self):
        try:
            self.unique_id = ubinascii.hexlify(unique_id()).decode('utf-8') 
            if(self.verbose): print(f"Unique ID: {self.unique_id}")
            
            
            return(self.unique_id)
        except Exception as e:
            print(f"Could not get unique ID....{e}")
    def setupInternalLed(self):
        try:
            self.led = Pin(self.onboardLedGPIO, Pin.OUT)
        except Exception as e:
            print(f"Error setting up on board LED...{e}")
    def blinkOnboardLed(self, reps=8):
        if(not self.led):
            self.setupInternalLed()
        state = True
        for _ in range(reps):
            time.sleep_ms(200)
            self.led.value(state)
            state = not state
        return(1)
    def getCpuTemp(self):
        try:
            self.CpuTemperature = esp32.raw_temperature()
            if(self.verbose): print(f"{esp32.raw_temperature()}", chr(176), "F")
        except Exception as e:
            self.CpuTemperature = -1
            print(f"Could not get temp off of CPU... {e}")
    def getMyMacAddr(self):
        try: 
            wlan_sta = network.WLAN(network.STA_IF)
            wlan_sta.active(True)
            self.wlanMacHex = wlan_sta.config('mac')
            self.wlanMacAscii = ubinascii.hexlify(self.wlanMacHex).decode()
            if(self.verbose): print(f"Loacl Mac Addr  (ASCII): {self.wlanMacAscii}")
            if(self.verbose): print(f"Loacl Mac Addr  (HEX): {self.wlanMacHex}")
            return(self.wlanMacHex)
        except Exception as e:
            print(f"Could not get mac address for board....{e}")
    def scanForWifiNetworks(self):
        sta_if = network.WLAN(network.STA_IF)
        sta_if.active(True)
        authmodes = ['Open', 'WEP', 'WPA-PSK' 'WPA2-PSK4', 'WPA/WPA2-PSK']
        for (ssid, bssid, channel, RSSI, authmode, hidden) in sta_if.scan():
          if ("Open" in authmodes[authmode]) and (ssid is not None):
              ssid = (ssid).decode("utf-8")
              if(self.verbose): print("Authmode open for:", ssid )
              return(ssid)
        if(self.verbose): print("No valid SSIDS found open")
        return(0)  
    def connectToWifi(self, ssid = None):
        tryLimit = 5
        station = network.WLAN(network.STA_IF)
        if station.isconnected() == True:
            ipAddr, netmask, router, dns = station.ifconfig()
            if(self.verbose): print("Already connected with IP Address: ", ipAddr)
            self.wifiStation = station
            return(ipAddr)
        if (ssid == None):
            ssid = self.scanForWifiNetworks()
        if(ssid != ""):

            station.config(channel=6)
            station.active(True)
            
            if(self.verbose): print("Not currently connected to Wifi, attempting connection to : ", ssid)
            station.connect(ssid)
            while not station.isconnected():	
                print(".", end="")
                time.sleep(1)  
            print("Connection Successful") 
            print(station.ifconfig())
            time.sleep(1)
            self.getNtpTime()
            self.wifiStation = station
            ipAddr, netmask, router, dns = station.ifconfig()
            print(f"ipAddr {ipAddr}, netmask {netmask}, router {router}, dns{dns}, channel {station.config("channel")}")
            self.wifiChannel = station.config("channel")
            return(ipAddr)
        else:
            if(self.verbose): print("Did not recieve open ssid to connect to.... Trying Again")
            attempts =0
            if(attempts < tryLimit ):
                self.connectToWifi()
                attempts +=1
            else:
                print(f"Unable to connect to wifi after {tryLimit} attempts, exiting program")
                return(0) 
    def scanI2C(self):
        i2c=machine.I2C(0,sda=machine.Pin(self.sdaPin), scl=machine.Pin(self.sclPin), freq=400000)
        if(self.verbose): print('Scanning I2C bus.')
        devices = i2c.scan() # this returns a list of devices
        device_count = len(devices)
        if device_count == 0:
            print('No i2c device found.')
        else:
            if(self.verbose): print(device_count, 'devices found.')
        for device in devices:
            if(self.verbose): print('Decimal address:', device, ", Hex address: ", hex(device))        
    def setupOLED(self):
        i2c = SoftI2C( scl=Pin(self.sclPin), sda=Pin(self.sdaPin))
        oled_width = 128
        oled_height = 64
        try:
            self.oled =  ssd1306Driver.SSD1306_I2C(oled_width, oled_height, i2c)
        except Exception as e:
            print(f"Could not connect to OLED... {e}")
        self.oled.fill(0)
        self.oled.text("Starting",0,0,1)
        self.oled.show()
        return(self.oled)

    def espnowInit(self):
        try:
            
            sta = network.WLAN(network.STA_IF)    
            sta.active(True)
            sta.disconnect() 
            sta.config(channel=5)
            if(self.verbose): print(f"espNow channel: {sta.config('channel')}")
            
            # sta.disconnect()        
            self.espNowObj = espnow.ESPNow()  
            self.espNowObj.active(True)
            if(self.verbose): print(f"espNow Init Status: {self.espNowObj}")
            if(self.oled):
                self.oled.text("EspNow Initalized", 0, 15, 1)
                self.oled.show()
            return(self.espNowObj)
        except Exception as e:
            print(f"Error setting up ESPNow Object in class....{e}")
            
    def espNowSendMessage(self,peer, msg):
        if(self.espNowObj == None):
            self.espnowInit()
        try:
            # self.wifiStation.disconnect()
            status= self.espNowObj.send(peer, msg , True)
            if(self.verbose): print(f"Sending data to Peer...{status}")
        except Exception as e:
            print(f"Error sending ESPNow Message....{e}")
            

    def espNowRecieve(self):
        if(self.espNowObj == None):
            self.espnowInit()
        try: 
            print("Setting up EspNow in Reciever Mode")
            while(True):
                host, msg = self.espNowObj.recv()
                if msg:             # msg == None if timeout in recv()
                    print(host, msg)
                    print(self.espNowObj.peers_table)
                    if(self.oled):
                        print("HI")
                        self.oled.fill(0)
                        self.oled.show()
                        self.oled.text(msg, 0, 20, 1)
                        self.oled.show()
                time.sleep_ms(100)
        except KeyboardInterrupt  as k:
            print(f"Keyboard Interupt....{k}")
        except Exception as e:
            print(f"Error recieving ESPNow Message....{e}")


device = ESP32_Wroom32Devkit1()
device.setupOLED()
device.getMyMacAddr()
# device.connectToWifi()
# device.espnowInit()
# ping('google.com')

# ping('google.com')


Loacl Mac Addr  (ASCII): a0b765587638
Loacl Mac Addr  (HEX): b'\xa0\xb7eXv8'


In [ ]:
device.espNowRecieve()

espNow channel: 5
espNow Init Status: <ESPNow object at 3ffe7430>
Setting up EspNow in Reciever Mode
b'\xa0\xb7eJ\x08\x98' b'HelloWORLD'
{b'\xa0\xb7eJ\x08\x98': [-19, 3826]}
HI
b'\xa0\xb7eJ\x08\x98' b'HelloWORLD'
{b'\xa0\xb7eJ\x08\x98': [-22, 4828]}
HI
b'\xa0\xb7eJ\x08\x98' b'HelloWORLD'
{b'\xa0\xb7eJ\x08\x98': [-23, 5831]}
HI
b'\xa0\xb7eJ\x08\x98' b'HelloWORLD'
{b'\xa0\xb7eJ\x08\x98': [-21, 6835]}
HI
b'\xa0\xb7eJ\x08\x98' b'HelloWORLD'
{b'\xa0\xb7eJ\x08\x98': [-23, 7837]}
HI
b'\xa0\xb7eJ\x08\x98' b'HelloWORLD'
{b'\xa0\xb7eJ\x08\x98': [-21, 8843]}
HI
b'\xa0\xb7eJ\x08\x98' b'HelloWORLD'
{b'\xa0\xb7eJ\x08\x98': [-23, 9846]}
HI
.b'\xa0\xb7eJ\x08\x98' b'HelloWORLD'
{b'\xa0\xb7eJ\x08\x98': [-22, 10849]}
HI
b'\xa0\xb7eJ\x08\x98' b'HelloWORLD'
{b'\xa0\xb7eJ\x08\x98': [-23, 11852]}
HI
b'\xa0\xb7eJ\x08\x98' b'HelloWORLD'
{b'\xa0\xb7eJ\x08\x98': [-22, 12855]}
HI
b'\xa0\xb7eJ\x08\x98' b'HelloWORLD'
{b'\xa0\xb7eJ\x08\x98': [-23, 13858]}
HI
b'\xa0\xb7eJ\x08\x98' b'HelloWORLD'
{b'\xa0\xb7eJ\x08\x98'

In [ ]:
if(None):
    print("true")

In [ ]:
# device.espNowRecieve()
